In [ ]:
import plotly.graph_objects as go
import dash
import plotly.express as px

import dash_html_components as html

import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.offline as pyo
import numpy as np
import pandas as pd
from os.path import exists

app  = dash.Dash()
country_codes = ['BR', 'CA', 'DE', 'FR', 'GB', 'IN', 'JP', 'KR', 'MX', 'RU', 'US']
category_ids_to_names_dict = {1: "Film & Animation",
2: "Autos & Vehicles",
10: "Music",
15: "Pets & Animals",
17: "Sports",
18: "Short Movies",
19: "Travel & Events",
20: "Gaming",
21: "Videoblogging",
22: "People & Blogs",
23: "Comedy",
24: "Entertainment",
25: "News & Politics",
26: "Howto & Style",
27: "Education",
28: "Science & Technology",
29: "Nonprofits & Activism",
30: "Movies",
31: "Anime/Animation",
32: "Action/Adventure",
33: "Classics",
34: "Comedy",
35: "Documentary",
36: "Drama",
37: "Family",
38: "Foreign",
39: "Horror",
40: "Sci-Fi/Fantasy",
41: "Thriller",
42: "Shorts",
43: "Shows",
44: "Trailers"}

category_names_to_ids_dict = {"Film & Animation":1,
"Autos & Vehicles":2,
"Music":10,
"Pets & Animals":15,
"Sports":17,
"Short Movies":18,
"Travel & Events":19,
"Gaming":20,
"Videoblogging":21,
"People & Blogs":22,
"Comedy":23,
"Entertainment":24,
"News & Politics":25,
"Howto & Style":26,
"Education":27,
"Science & Technology":28,
"Nonprofits & Activism":29,
"Movies":30,
"Anime/Animation":31,
"Action/Adventure":32,
"Classics":33,
"Comedy":34,
"Documentary":35,
"Drama":36,
"Family":37,
"Foreign":38,
"Horror":39,
"Sci-Fi/Fantasy":40,
"Thriller":41,
"Shorts":42,
"Shows":43,
"Trailers":44}

dates_from_processed_dataset = pd.read_csv("processed_dataset.csv", usecols=[7])
dates_from_processed_dataset.sort_values('trending_date')
unique_dates = dates_from_processed_dataset.trending_date.unique()
unique_dates = np.array(unique_dates)
no_of_unique_dates = len(unique_dates)

country_list = []
for i in country_codes:
    country_list.append({'label': i, 'value': i})

categories_list = []
for (key, value) in category_ids_to_names_dict.items():
    categories_list.append({'label': value, 'value': value})

current_data_set = "Dataset/Titledata/BR/BR_title_totals.csv"
df_test  = pd.read_csv(current_data_set)

total_titles = np.array(df_test.total_number_of_titles)[0]
did_use_par_or_bracks = (np.array(df_test.number_of_titles_with_parenthesis_or_squarebracket_usage)[0]/total_titles)*100
did_use_caps = (np.array(df_test.number_of_titles_with_caps_usage)[0]/total_titles)*100
did_use_emojis = (np.array(df_test.number_of_titles_with_emoji_usage)[0]/total_titles)*100

did_not_use_par_or_bracks = 100 - did_use_par_or_bracks
did_not_use_caps = 100 - did_use_caps
did_not_use_emojis = 100 - did_use_emojis

x = ['Did use () or []', 'Did use CAPS', 'Did use emojis']

fig = go.Figure()
fig.add_trace(go.Bar(name="Yes", x=x, y=[did_use_par_or_bracks, did_use_caps, did_use_emojis], marker_color='rgb(44, 127, 184)'))
fig.add_trace(go.Bar(name="No", x=x, y=[did_not_use_par_or_bracks, did_not_use_caps, did_not_use_emojis], marker_color='rgb(254, 178, 76)'))

fig.update_layout(yaxis_title="Percentage of videos",)
fig.update_layout(title_text='Titles')
fig.update_layout(barmode='stack')
fig.update_layout(yaxis_range=(0, 100))

app.layout = html.Div(children=[
    html.Div(
        dcc.Dropdown(
            id="title_drop_down",
            options=country_list,
            value=['BR'],
            multi=True
        ),
        style={"text-align": "center", "display": "inline-block", "width":"100%"}),
    html.Div(
        dcc.Dropdown(id="categories_drop_down", options=categories_list, value=[], multi=True),
             style={"text-align": "center", "display": "inline-block", "width":"100%"}),
    html.Div(dcc.Graph(id='title_bar_chart',figure=fig),
             style={"text-align": "center", "display": "inline-block", "width":"100%"}),
    html.Div(dcc.RangeSlider(
            id="dataslider",
            min=0,
            max=no_of_unique_dates,
            updatemode='mouseup',
            step=1,
            value=[0, no_of_unique_dates],
            pushable=1), style={"text-align": "center", "display": "inline-block", "width":"100%"}),
     html.Div(id='my-output', style={"text-align": "center", "display": "inline-block", "width":"100%"})
            ], style={"width": "800px", "border": "1px solid black"})


def update_data(input_countries, input_categories, slider_interval):

    #Should not use global - But only if multi-user session according to the tutorials!
    global current_data_set, df_test, total_titles, did_use_par_or_bracks, did_use_caps, did_use_emojis, did_not_use_par_or_bracks, did_not_use_caps, did_not_use_emojis

    total_titles = 0
    did_use_par_or_bracks = 0
    did_use_caps = 0
    did_use_emojis = 0
    did_not_use_par_or_bracks = 0
    did_not_use_caps = 0
    did_not_use_emojis = 0

    for input_country in input_countries:
        if len(input_categories) == 0:
            if slider_interval[0] == 0 and slider_interval[1] == no_of_unique_dates:
                current_data_set = "Dataset/Titledata/" + input_country + "/" + input_country + "_title_totals.csv"
                df_test  = pd.read_csv(current_data_set)

                total_titles += np.array(df_test.total_number_of_titles)[0]
                did_use_par_or_bracks += np.array(df_test.number_of_titles_with_parenthesis_or_squarebracket_usage)[0]
                did_use_caps += np.array(df_test.number_of_titles_with_caps_usage)[0]
                did_use_emojis += np.array(df_test.number_of_titles_with_emoji_usage)[0]
            else:
                current_data_set = "Dataset/Titledata/" + input_country + "/" + input_country + "_allcategories_totals_per_day.csv"
                df_test  = pd.read_csv(current_data_set)
                mask = (df_test['date'] >= unique_dates[slider_interval[0]]) & (df_test['date'] <= unique_dates[slider_interval[1]-1])
                df_test = df_test.loc[mask]

                for index, row in df_test.iterrows():
                    total_titles += row['total_number_of_titles']
                    did_use_par_or_bracks += row['number_of_titles_with_parenthesis_or_squarebracket_usage']
                    did_use_caps += row['number_of_titles_with_caps_usage']
                    did_use_emojis += row['number_of_titles_with_emoji_usage']
        else:
            for input_category in input_categories:
                category_id = category_names_to_ids_dict[input_category]
                current_data_set = "Dataset/Titledata/" + input_country + "/" + input_country + "_category" + str(category_id) + "_totals_per_day.csv"
                file_exists = exists(current_data_set)
                if file_exists:
                    df_test  = pd.read_csv(current_data_set)
                else:
                    continue

                mask = (df_test['date'] >= unique_dates[slider_interval[0]]) & (df_test['date'] <= unique_dates[slider_interval[1]-1])
                df_test = df_test.loc[mask]

                for index, row in df_test.iterrows():
                    total_titles += row['total_number_of_titles']
                    did_use_par_or_bracks += row['number_of_titles_with_parenthesis_or_squarebracket_usage']
                    did_use_caps += row['number_of_titles_with_caps_usage']
                    did_use_emojis += row['number_of_titles_with_emoji_usage']
                #total_titles += np.array(df_test.total_number_of_titles)[0]
                #did_use_par_or_bracks += np.array(df_test.number_of_titles_with_parenthesis_or_squarebracket_usage)[0]
                #did_use_caps += np.array(df_test.number_of_titles_with_caps_usage)[0]
                #did_use_emojis += np.array(df_test.number_of_titles_with_emoji_usage)[0]
    did_use_par_or_bracks = (did_use_par_or_bracks/total_titles)*100 if total_titles > 0 else 0
    did_use_caps = (did_use_caps/total_titles)*100 if total_titles > 0 else 0
    did_use_emojis = (did_use_emojis/total_titles)*100 if total_titles > 0 else 0

    did_not_use_par_or_bracks = 100 - did_use_par_or_bracks if total_titles > 0 else 0
    did_not_use_caps = 100 - did_use_caps if total_titles > 0 else 0
    did_not_use_emojis = 100 - did_use_emojis if total_titles > 0 else 0

@app.callback(
    Output(component_id='title_bar_chart', component_property='figure'),
    Input("title_drop_down", "value"),
    Input("categories_drop_down", "value"),
    Input("dataslider", "value"))
def update_output_div(input_countries, input_categories, slider_interval):
    update_data(input_countries, input_categories, slider_interval)

    fig = go.Figure()
    fig.add_trace(go.Bar(name="Yes", x=x, y=[did_use_par_or_bracks, did_use_caps, did_use_emojis], marker_color='rgb(44, 127, 184)'))
    fig.add_trace(go.Bar(name="No", x=x, y=[did_not_use_par_or_bracks, did_not_use_caps, did_not_use_emojis], marker_color='rgb(254, 178, 76)'))

    fig.update_layout(title_text='Titles')
    fig.update_layout(barmode='stack')
    fig.update_layout(yaxis_range=(0, 100))
    fig.update_layout(transition={
                'duration': 500,
                'easing': 'cubic-in-out'
        })

    return fig

@app.callback(
    Output('my-output', component_property='children'),
    Input('dataslider', 'value'))
def settext(slider_interval):
    return unique_dates[slider_interval[0]] + " - " + unique_dates[slider_interval[1]-1]

if __name__ == '__main__' :
    app.run_server()


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Nov/2021 22:20:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 22:20:21] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 22:20:21] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 22:20:21] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Nov/2021 22:20:21] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Nov/2021 22:20:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 22:20:21] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Nov/2021 22:20:21] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Nov/2021 22:20:21] "POST /_dash-update-component HTTP/1.1" 200 -


85593
6554


127.0.0.1 - - [17/Nov/2021 22:20:23] "POST /_dash-update-component HTTP/1.1" 200 -


0
0


...

5784
1273


127.0.0.1 - - [17/Nov/2021 22:29:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 22:29:25] "POST /_dash-update-component HTTP/1.1" 200 -


6629
1388


127.0.0.1 - - [17/Nov/2021 22:29:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2021 22:29:27] "POST /_dash-update-component HTTP/1.1" 200 -


448
74
